    Covid --- 0
    Normal --- 1

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os
import time
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import *
from sklearn.metrics import *
from tensorflow.keras.models import load_model


In [ ]:
train_tfrecord = 'XRay_train.tfrecords'
test_tfrecord = 'XRay_test.tfrecords'
train_percentage = 0.8  # Proportion of training set

random.seed(20)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
input_path='drive/MyDrive/COVID-19_Radiography_Dataset/'

In [ ]:
learning_rate = 0.001
buffer_size = 512
batch_size = 32
epochs = 20

img_size = 224

**Load Data**

In [ ]:
def read_directory():
    data_filenames = []
    data_labels = []

    for filename in os.listdir(input_path + 'COVID'):
        data_filenames.append(input_path + 'COVID/' + filename)
        data_labels.append(0)

   
    for filename in os.listdir(input_path + 'Normal'):
        data_filenames.append(input_path + 'Normal/' + filename)
        data_labels.append(1)

        
    data_size = len(data_labels)

    tmp_uni = list(zip(data_filenames, data_labels))

    random.shuffle(tmp_uni)

    train_size = int(data_size * train_percentage)
    print('Size of training set：', train_size)
    print('Size of test set：', data_size - train_size)

    train_list = tmp_uni[0:train_size]
    test_list = tmp_uni[train_size:]

    train_filenames, train_labels = zip(*train_list)
    test_filenames, test_labels = zip(*test_list)

    return train_filenames, train_labels, test_filenames, test_labels

**Build TFRecord**

In [ ]:
def build_train_tfrecord(train_filenames, train_labels):  # Generate TFRecord of training set 
    with tf.io.TFRecordWriter(train_tfrecord)as writer:
        for filename, label in zip(train_filenames, train_labels):
            image = open(filename, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),  # img > Bytes
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  # label > Int
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())


def build_test_tfrecord(test_filenames, test_labels):  # Generate TFRecord of test set
    with tf.io.TFRecordWriter(test_tfrecord)as writer:
        for filename, label in zip(test_filenames, test_labels):
            image = open(filename, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

**Decode TFRecord and get data**

In [ ]:
def _parse_example(example_string):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_png(feature_dict['image'], channels=3)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [img_size, img_size]) / 255.0
    return feature_dict['image'], feature_dict['label']


def get_train_dataset(train_tfrecord):  # read TFRecord
    raw_train_dataset = tf.data.TFRecordDataset(train_tfrecord)
    train_dataset = raw_train_dataset.map(_parse_example)

    return train_dataset


def get_test_dataset(test_tfrecord):
    raw_test_dataset = tf.data.TFRecordDataset(test_tfrecord)
    test_dataset = raw_test_dataset.map(_parse_example)

    return test_dataset


def data_Preprocessing(train_dataset, test_dataset):
    train_dataset = train_dataset.shuffle(buffer_size)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    test_dataset = test_dataset.batch(batch_size)
    test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return train_dataset, test_dataset

**Model**

In [ ]:
def InceptionResNetV2_model():
    incp_res_v2 = tf.keras.applications.InceptionResNetV2(weights='imagenet',include_top=False, input_shape=[img_size,img_size,3])
    incp_res_v2.trainable= True
    model = tf.keras.Sequential([
        incp_res_v2,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )
    
    return model

**Train&Test**

In [ ]:
def scheduler(epoch,lr):
    if epoch<8:
        return lr
    else:
        return lr*0.9
callback = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)


In [ ]:
def train():
    time_start = time.time()
    
    model = InceptionResNetV2_model()
    
    model.summary()
    
    train_history = model.fit(train_dataset, epochs=20, callbacks=[callback])

    model.save('mymodel.h5')
    
    print('Model saved.')
    
    time_end = time.time()
    print('Training Time:', time_end - time_start)
    print('\n')

    return train_history

In [ ]:
def show_train_history(train_history, index):
    plt.plot(train_history.history[index])
    plt.title('Train History')
    plt.ylabel(index)
    plt.xlabel('Epoch')
    plt.show()

In [ ]:
def test(test_labels):
    test_labels = np.array(test_labels)
    model = load_model('mymodel.h5')
    
    print('Testing:')
    
    model.evaluate(test_dataset)
    
    predIdxs = model.predict(test_dataset)
    predIdxs = np.argmax(predIdxs, axis=1) 

    target_names = ['COVID', 'NORMAL']
    print('\n')
    print(classification_report(test_labels, predIdxs, target_names=target_names, digits=5))

**Run Now¶**

In [ ]:
if __name__ == "__main__":
    train_filenames, train_labels, test_filenames, test_labels = read_directory()

    build_train_tfrecord(train_filenames, train_labels)
    build_test_tfrecord(test_filenames, test_labels)

    train_dataset = get_train_dataset(train_tfrecord)
    test_dataset = get_test_dataset(test_tfrecord)

    print('Info of train_dataset', type(train_dataset))
    print('Info of test_dataset', type(test_dataset))

    train_dataset, test_dataset = data_Preprocessing(train_dataset, test_dataset) 

    train_history = train()
    
    test(test_labels)
    
    show_train_history(train_history, 'sparse_categorical_accuracy')

In [ ]:
model=load_model('mymodel.h5')

In [ ]:
import tensorflow as tf

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

In [ ]:
with open('Covid19.tflite', 'wb') as f:
  f.write(quantized_tflite_model)